One of my strategies in this competition was to train segmentation models using synthetic signals added only to control images from train and test.


A model trained only using that simulated targets scored Public LB: 0.77128 and Private: 0.76992, showing that simulated signal generator matches, not perfectly, but very well the signals injected by hosts.


To generate the simulated masks I was based in 2017 SETI competition signal generator.
The signals are:


0 - Straight Line


1 - Straight Line + Sine


2 - Straight Line + random jitter + randon intensity.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 12]

np.random.seed(1111)

In [ ]:
def ret_freq(A=5, F=1.):
    time = np.arange(6*273)
    F = 1 + 0.667*(np.random.rand()-0.5)
    K = 2 * 3.14159 * np.random.rand()
    freq = A * np.random.rand() * np.sin(2*3.14156*6*F*time/(273*6) + K)
    return freq.astype(np.float32)

def ret_line():
    k = 8 + np.random.randint(256-16)
    l = (np.random.rand()-0.5) * 0.075
    a = np.random.rand() * 0.002 * l
    a = np.cumsum(a*np.ones(6*273))
    res = np.clip( k + np.cumsum((l+a)*np.ones(6*273)) , 0 , 255 )
    return res.astype(np.float32)

def aug_target(target=0):
    tmax = 0
    while tmax <= 0:
        tgt = np.zeros( (6*273,256), dtype=np.float32)
        
        if target==0: # Straight Line + Sine
            sls = ret_freq(A=10, F=1.) + ret_line()

            ini = max([0,np.random.randint(273+136) - 136])
            fim = min( [5*273, 4*273+np.random.randint(273+136)] )
            for t in range(ini,fim):
                py = int(np.round(sls[t]))
                if (py<=1)or(py>=254):
                    continue
                a = sls[t] - py
                if a>=0:
                    a = 1-a
                    b = 1-a
                    tgt[t,py] = a + np.random.normal(0, 0.1)
                    tgt[t,py+1] = b + np.random.normal(0, 0.1)
                else:
                    a = 1+a
                    b = 1-a
                    tgt[t,py] = a + np.random.normal(0, 0.1)
                    tgt[t,py-1] = b + np.random.normal(0, 0.1)
                   
        elif target==1:# Straight Line
            sls = ret_line()

            ini = max([0,np.random.randint(273+136) - 136])
            fim = min( [5*273, 4*273+np.random.randint(273+136)] )
            for t in range(ini,fim):
                py = int(np.round(sls[t]))
                if (py<=1)or(py>=254):
                    continue
                a = sls[t] - py
                if a>=0:
                    a = 1-a
                    b = 1-a
                    tgt[t,py] = a + np.random.normal(0, 0.1)
                    tgt[t,py+1] = b + np.random.normal(0, 0.1)
                else:
                    a = 1+a
                    b = 1-a
                    tgt[t,py] = a + np.random.normal(0, 0.1)
                    tgt[t,py-1] = b + np.random.normal(0, 0.1)
                    
        elif target==2: # Straight Line + random jitter + randon intensity.
            for n in [0,2,4]:
                I = np.random.normal(0,0.05)-0.95
                K = np.random.randint(256)
                A = 2.0*np.random.rand()
                
                ini = (n*273) + np.random.randint(273)
                fim = ini + 16 + np.random.randint(136+136//2)
                for t in range(ini, fim):
                    if np.random.normal() >= I:
                        py = int(np.round(K))
                        if (py<=0)or(py>=255):
                            continue
                        tgt[t,py] = np.random.normal(1, 0.1)
                    else:
                        K = K + np.random.normal(0,A)
                    
        elif target==3:  # Straight Line + random jitter + randon intensity.                       
            K = np.random.randint(256)
            for n in [0,2,4]:
                I = np.random.normal(0,0.05)-1.95
                
                ini = (n*273) + np.random.randint(273)
                fim = ini + 16 + np.random.randint(64)
                for t in range(ini, fim):
                    if np.random.normal() >= I:
                        py = int(np.round(K))
                        if (py<=0)or(py>=255):
                            continue
                        tgt[t,py] = np.random.normal(1, 0.1)
                    
        elif target==4: # Bright Dot ??? 
            for n in range(5):
                if np.random.rand()>0.5:
                    K = np.random.choice([0,2,4])*273 + np.random.randint(273) + 1
                    Y = np.random.randint(256-2)+1
                    tgt[K,Y] = np.random.normal(1, 0.1) 
                    tgt[K,Y+1] = np.random.normal(1, 0.1) 
                    tgt[K+1,Y] = np.random.normal(1, 0.1) 
                    tgt[K,Y-1] = np.random.normal(1, 0.1) 
                    tgt[K-1,Y] = np.random.normal(1, 0.1) 
            

        tgt = np.stack((tgt[:273,:],tgt[2*273:3*273,:],tgt[4*273:5*273,:]))
        tgt = tgt.astype(np.float32)
        
        tmax = tgt.max()
        if tmax>0:
            tgt /= tmax
        
    return tgt

In [ ]:
for i in range(8):
    img = aug_target(0)
    print( img.shape)

    img = np.vstack( img )
    plt.imshow(np.rot90(img))
    plt.show()

In [ ]:
for i in range(8):
    img = aug_target(1)
    print( img.shape)

    img = np.vstack( img )
    plt.imshow(np.rot90(img))
    plt.show()

In [ ]:
for i in range(8):
    img = aug_target(2)
    print( img.shape)

    img = np.vstack( img )
    plt.imshow(np.rot90(img))
    plt.show()

# How to use the signal masks

In [ ]:
import glob

files = glob.glob('../input/seti-breakthrough-listen/train/0/*.npy')
len(files)

In [ ]:
# Load an image
image = np.load(files[132]).astype(np.float32)
image = image[[1,3,5]].copy()

plt.imshow(np.rot90(np.vstack(image)))

In [ ]:
# Build a random signal mask
mask = aug_target(0)

plt.imshow(np.rot90(np.vstack(mask)))

In [ ]:
# Apply to the image

for i in range(3):
    S = np.max([0.1,(np.random.normal(0.75,0.1))])*image[i].std()
    print(S)
    image[i] += S*mask[i]
    image[i] /= image[i].std()

plt.imshow(np.rot90(np.vstack(image)))

# Now you can train segmentation models

In [ ]:
plt.imshow( 
    np.vstack((
        np.rot90(np.vstack(image/image.max())),
        np.rot90(np.vstack(mask)),
    ))
)